In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
! pip install transformers sentencepiece

In [3]:
# !git lfs install
# !git clone https://huggingface.co/cointegrated/LaBSE-en-ru

In [4]:
import random
import numpy as np
import torch
from torch import nn
from collections import OrderedDict
from transformers import AutoTokenizer, AutoModel

In [5]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed = 42
set_seed(seed)

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
        self.base = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
        n_dim = 312
        self.head = nn.Sequential(OrderedDict( [('fc_1' , nn.Linear(n_dim, n_dim*2)),
                                                ('relu_1' , nn.GELU()),
                                                ('layernorm_1' , nn.LayerNorm(n_dim*2, eps=1e-12)),
                                                ('fc_2' , nn.Linear(n_dim*2, n_dim)),
                                                ('relu_2' , nn.GELU()),
                                                ('dropout', torch.nn.Dropout(.0)),
                                                ('layernorm_2' , nn.LayerNorm(n_dim, eps=1e-12)),
                                                ('fc_3' , nn.Linear(n_dim, 2, bias=False)),
                                                ('softmax', nn.Softmax(dim=-1))
                    ]))

    def forward(self, text):
        token = self.tokenizer(text, padding=True, truncation=True, return_tensors='pt').to(device)
        model_output = self.base(**token)
        result = self.head(model_output.pooler_output)
        return result

In [8]:
# m = Model()
# m.forward('asd')